In [3]:
import json
import datetime
import time
import yaml
import pandas as pd
import mojito
import pprint
import requests

In [2]:
with open('C:/Users/cc843/Desktop/辛旻宗/주식프로젝트/key/config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']

In [4]:
def get_access_token():
    """토큰 발급"""
    headers = {"content-type": "application/json"}
    body = {"grant_type": "client_credentials", "appkey": APP_KEY, "appsecret": APP_SECRET}
    PATH = "oauth2/tokenP"
    URL = f"{URL_BASE}/{PATH}"
    res = requests.post(URL, headers=headers, data=json.dumps(body))
    ACCESS_TOKEN = res.json()["access_token"]
    return ACCESS_TOKEN

In [5]:
ACCESS_TOKEN = get_access_token()

In [6]:
broker = mojito.KoreaInvestment(
    api_key=APP_KEY,
    api_secret=APP_SECRET,
    acc_no="50101465-01"
)

resp = broker.fetch_ohlcv(
    symbol="005930",
    timeframe='D',
    adj_price=True,
    start_day="20200101",
    end_day="20221230" # 정확히는 모르겠는데 end_day를 기준으로 100개를 가져옴
)

pprint.pprint(resp)

{'msg1': '정상처리 되었습니다.',
 'msg_cd': 'MCA00000',
 'output1': {'acml_tr_pbmn': '2095413492000',
             'acml_vol': '26608944',
             'askp': '78900',
             'bidp': '78800',
             'cpfn': '7780',
             'eps': '2131.00',
             'hts_avls': '4710158',
             'hts_kor_isnm': '삼성전자',
             'itewhol_loan_rmnd_ratem name': '0.13',
             'lstn_stcn': '5969782550',
             'pbr': '1.52',
             'per': '37.02',
             'prdy_ctrt': '-0.50',
             'prdy_vol': '44569800',
             'prdy_vrss': '-400',
             'prdy_vrss_sign': '5',
             'prdy_vrss_vol': '-17960856',
             'stck_fcam': '100',
             'stck_hgpr': '79900',
             'stck_llam': '55600',
             'stck_lwpr': '77800',
             'stck_mxpr': '103000',
             'stck_oprc': '79600',
             'stck_prdy_clpr': '79300',
             'stck_prdy_hgpr': '79300',
             'stck_prdy_lwpr': '77700',
             

In [7]:
df = pd.DataFrame(resp['output2'])
dt = pd.to_datetime(df['stck_bsop_date'], format="%Y%m%d")
df.set_index(dt, inplace=True)
df = df[['stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_clpr']]
df.columns = ['open', 'high', 'low', 'close']
df.index.name = "date"
df

,open,high,low,close
date,,,,
2022-12-29,56000,56200,55300,55300
2022-12-28,57600,57600,56400,56600
2022-12-27,58000,58400,57900,58100
2022-12-26,58000,58100,57700,57900
2022-12-23,58200,58400,57700,58100
...,...,...,...,...
2022-08-11,59600,60000,59300,59900
2022-08-10,58900,59200,58600,59100
2022-08-09,60600,60700,59600,60000
